In [1]:
import numpy as np
from matplotlib import pyplot as plt

from module_.dataLoader import dataLoader
from module_.featureExtraction import feature_extraction
from module_.changePointDetection import change_point_detection

from module_.info.config import feature_name

In [2]:
episodes, transitions, labels = dataLoader("adlmr")

In [ ]:
test_index = 2
episode, transition, label = episodes[test_index], transitions[test_index], labels[test_index]

sensor_list = sorted(set(episode[:,0]))
segmentation_features = np.array(feature_extraction(episode, "adlmr", sensor_list))

score_s = np.array(change_point_detection(segmentation_features, "RuLSIF"))

fig = plt.figure(figsize=(20, 1))
ax = fig.add_subplot(1, 1, 1)

plt.title(label)
plt.ylim(0, 1)
plt.plot(range(len(score_s)), score_s)
plt.axvline(x=transition, color="g", linestyle=":")
plt.axhline(y=0.1, color="r", linestyle=":")

# test on group activity dataset
# statistics of false alarms per activity: (hh101, adlmr, testbed)
# is it problem or not?

In [ ]:
score_list = []
for ei in range(len(episodes)):
    if ei%100==0:
        print(ei, len(episodes))
    test_index = ei
    episode, transition, label = episodes[test_index], transitions[test_index], labels[test_index]

    sensor_list = sorted(set(episode[:,0]))
    segmentation_features = np.array(feature_extraction(episode, "adlmr", sensor_list))

    scores = np.array(change_point_detection(segmentation_features, "RuLSIF"))
    score_list.append(scores)

np.save("./replication/adlmr/scores_r.npy", score_list)

In [ ]:
true_positive = true_negative = false_positive = false_negative = 0

threshold = 0.1

for ei in range(len(episodes)):
    test_index = ei
    episode, transition, label = episodes[test_index], transitions[test_index], labels[test_index]
    scores = score_list[test_index]

    gt_timestamp_l = float(episode[transition-1][2])
    gt_timestamp_r = float(episode[transition][2])

    tp = tn = fp = fn =0
    activation = False
    activation_timestamp = 0
    for j in range(len(episode)):
        s = scores[j]

        score = s
        timestamp = float(episode[j][2])

        if score>=threshold: #POSITIVE
            # if activation and timestamp - activation_timestamp <=20 : #SKIP
            #     if j!=transition:
            #         tn+=1
            #     else:
            #         fn+=1
            #     activation = False
            #     continue
            if j==transition:
                tp+=1
            else:
                if abs(gt_timestamp_l-timestamp)<=2 or abs(gt_timestamp_r-timestamp)<=2:
                    tp+=1
                else:
                    fp+=1
            activation = True
            activation_timestamp = timestamp
        else:
            if j!=transition:
                tn+=1
            else:
                fn+=1

    true_positive+=tp
    true_negative+=tn
    false_positive+=fp
    false_negative+=fn

In [ ]:
true_positive / (true_positive+false_negative), false_positive / (false_positive+true_negative)

In [ ]:
label_list = set()
for label in labels:
    left, right = label.split("-")
    label_list.add(left); label_list.add(right)
label_list = sorted(label_list)

events_dict = {label:(0, 0) for label in label_list}
sensor_dict = {label:(0, 0) for label in label_list}
duration_dict = {label:(0, 0) for label in label_list}
alarm_dict = {label:(0, 0) for label in label_list}

for ei in range(len(episodes)):
    if ei%100==0:
        print(ei, len(episodes))
    test_index = ei
    episode, transition, label = episodes[test_index], transitions[test_index], labels[test_index]

    # left
    leftlabel = label[:2]
    events_dict[leftlabel]=(events_dict[leftlabel][0]+len(episode[:transition]),events_dict[leftlabel][1]+1)
    sensor_dict[leftlabel]=(sensor_dict[leftlabel][0]+len(set(episode[:transition,0])),sensor_dict[leftlabel][1]+1)
    duration_dict[leftlabel]=(duration_dict[leftlabel][0]+float(episode[transition-1,2])-float(episode[0,2]),duration_dict[leftlabel][1]+1)

    sensor_list = sorted(set(episode[:,0]))
    segmentation_features = np.array(feature_extraction(episode, "adlmr", sensor_list))

    scores = np.array(change_point_detection(segmentation_features[:transition+2], "RuLSIF"))

    alarm_dict[leftlabel]=(alarm_dict[leftlabel][0]+sum([1 for i in range(len(scores[:transition])) if scores[i]>0.1]),alarm_dict[leftlabel][1]+1)

In [ ]:
for k, v in alarm_dict.items():
    print(k, v[0]/v[1])

# 1. Dataloader - adlmr
# 2. Extract Features and Store them
# 3. Scoring (RuLSIF)
# 4. Select Feature set whose score is above threshold
# 5. analyze the change of Features and infer the cause that brought high score!

In [ ]:
threshold, window_size = 1e-1, 30

for i in range(len(episodes)):
    episode, transition, label = episodes[i], transitions[i], labels[i]

    sensor_list = sorted(set(episode[:,0])); print(len(sensor_list))

    features = np.array(
        feature_extraction(episode, "adlmr", sensor_list)
    )

    scores = np.array(
        change_point_detection(features, "RuLSIF")
    )

    positive_indices = [i for i in range(len(scores)) if scores[i]>threshold]

    break

In [ ]:
# print(len(positive_indices), len(scores))
print(positive_indices, transition)

test = 85 #positive_indices[1]
print(test, scores[test])

event_stream = episode[max(0, test-window_size+1):min(len(episode), test+3)]
feature_set = features[test:min(len(episode), test+3)]
# print(event_stream)

for f in range(feature_set.shape[1]):
    f_col = feature_set[:,f]

    
    if f>len(feature_name)-2:
        idx = int((f-len(feature_name)+2)/len(sensor_list))
        name = feature_name[len(feature_name)-2+idx] + sensor_list[(f-len(feature_name)+2)%len(sensor_list)]
    else:
        name = feature_name[f]
    if min(f_col)!=max(f_col):
        print(name, f_col)